In [144]:
# import des librairies requises
import pandas as pd
import re
from nltk.tokenize import sent_tokenize, word_tokenize
import nltk
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import CountVectorizer
nltk.download('punkt')
from stop_words import get_stop_words

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Antoine\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [145]:
# lecture des différents csv dans les 3 variables 
df1 = pd.read_csv("./../data/offers.csv")
df2 = pd.read_csv("./../data/organizations.csv")
df3 = pd.read_csv("./../data/recruiters.csv")

# df1
# df2
# df3

In [148]:
# on ajoute <p> comme expression régulière que l'on stocke dans prog
#prog = re.compile('<p>')

In [149]:
#
#result = prog.match('saved_column')
#bool(result)

False

In [168]:
#On compile les balises html pour pouvoir les soustraire avec regex.sub
#regex = re.compile('(<p>)(</p>)')
#regex.sub("", str(saved_column))

In [151]:
# Remplacer les valeurs NA avec ""

df1['description'] = [BeautifulSoup(text, "html.parser").get_text() for text in df1['description'].fillna("")]
cleaned_description = df1['description']

In [152]:
# Vectorisation

from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_ngrams = TfidfVectorizer(
    ngram_range=(1, 2), analyzer="word", binary=False)


In [153]:
#tfidf fit de cleaned_description

tfidf_ngrams.fit(cleaned_description)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [154]:
#stop words fr dans stop_words
stop_wordsFR = get_stop_words('french')

In [155]:
#création du CounVectorizer
vectorizer = CountVectorizer(stop_words = stop_wordsFR, tokenizer = word_tokenize)
vectorizer

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None,
        stop_words=['a', 'ai', 'aie', 'aient', 'aies', 'ait', 'alors', 'as', 'au', 'aucun', 'aura', 'aurai', 'auraient', 'aurais', 'aurait', 'auras', 'aurez', 'auriez', 'aurions', 'aurons', 'auront', 'aussi', 'autre', 'aux', 'avaient', 'avais', 'avait', 'avant', 'avec', 'avez', 'aviez', 'avions', 'avoir', '...ça', 'étaient', 'étais', 'était', 'étant', 'état', 'étiez', 'étions', 'été', 'étés', 'êtes', 'être'],
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=<function word_tokenize at 0x09133468>, vocabulary=None)

In [169]:
#décommenter pour visualiser de cleaned_description
#cleaned_description

In [170]:
#liste des stop words
#stop_words

In [163]:
#Y = vectorizer.fit_transform(filtered_sentence)
X = vectorizer.fit_transform(cleaned_description)

In [165]:
analyze = vectorizer.build_analyzer()

In [166]:
FinalDF = pd.DataFrame(data=vectorizer.transform(X).todense(),
columns=vectorizer.get_feature_names(),
index=X.index)

AttributeError: lower not found

In [175]:
xArray = X[:10].toarray()

In [176]:
xArray

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [177]:
vectorizer.vocabulary_.get('CDI')